In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print()
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
class PasVocDataset(torch.utils.data.Dataset):
    def __init__(self,im)

In [ ]:
import torch.nn as nn
import torch.functional as F
import torch

architecture_config = [
    (7, 64, 2, 3),
    "M",
    (3, 192, 1, 1),
    "M",
    (1, 128, 1, 0),
    (3, 256, 1, 1),
    (1, 256, 1, 0),
    (3, 512, 1, 1),
    "M",
    [(1, 256, 1, 0), (3, 512, 1, 1), 4],
    (1, 512, 1, 0),
    (3, 1024, 1, 1),
    "M",
    [(1, 512, 1, 0), (3, 1024, 1, 1), 2],
    (3, 1024, 1, 1),
    (3, 1024, 2, 1),
    (3, 1024, 1, 1),
    (3, 1024, 1, 1),
]

In [ ]:
class ConvBlock(nn.Module):
  def __init__(self,in_channels,out_channels,**kwargs):
    super(ConvBlock,self).__init__()
    self.conv = nn.Conv2d(in_channels=in_channels,out_channels=out_channels,**kwargs)
    self.bn = nn.BatchNorm2d(out_channels)
    self.relu = nn.LeakyReLU(0.1)

  def forward(self,x):
    x = self.conv(x)
    x = self.bn(x)
    x = self.relu(x)
    return x 



In [ ]:
class Yolov1(nn.Module):
  def __init__(self,in_channel,out_channel,**kwargs):
    super(Yolov1,self).__init__()
    self.in_channes = 3
    self.architecture = architecture_config
    self.body = self.create_body(self.architecture)
    self.fc = self.create_fc(**kwargs)

  def forward(self,x):
    x = self.body(x)
    x = torch.flatten(x,start_dim=1)
    x = self.fc(x)
    return x

  def create_body(self,architecture):
    layers = []
    for arch in architecture:
      if type(arch) == tuple:
        layers += [ConvBlock(in_channels =  self.in_channels,out_channels = arch[1],kernel_size = arch[0],stride = arch[2],padding = arch[3])]
        self.in_channels = arch[1]
       
      elif type(arch) == str:
        layers += [nn.MaxPool2d(stride=2,kernel_size=2)]
        self.in_channels /= 2

      elif type(arch) == list:
        for ar in range(arch[-1]):
          layers += [ConvBlock(in_channels = self.in_channels,out_channels = arch[0][1],kernel_size = arch[0][0],stride = arch[0][2],padding = arch[0][3])] 
          self.in_channels = arch[0][1]
          layers += [ConvBlock(in_channels = self.in_channels,out_channels = arch[1][1],kerner_size = arch[1][0],stride = arch[1][2],padding = arch[1][3])]
          self.in_channels = arch[1][1]
    return nn.Sequential(*layers)
  
  def create_fc(self,S,B,C):
    return nn.Sequential(
        nn.Flatten(),
        nn.Linear(S*S*1024,4096),
        nn.LeakyReLU(0.0), 
        nn.Dropout(0.0),
        nn.Linear(4096,S*S*(C+B*5))       
    )

In [ ]:
def iOU(pred,target,format='mid'):
    if format == 'mid':
        box_x1 = pred[...,0:1]-pred[...,2:3]/2
        box_x2 = pred[...,0:1]+pred[...,2:3]/2
        box_y1 = pred[...,1:2]-pred[...,3:4]/2
        box_y2 = pred[...,1:2]+pred[...,3:4]/2
        
        box_x1_ = target[...,0:1]-target[...,2:3]/2
        box_x2_ = target[...,0:1]+target[...,2:3]/2
        box_y1_ = target[...,1:2]-target[...,3:4]/2
        box_y2_ = target[...,1:2]+target[...,3:4]/2
        
    x1=torch.max(box_x1,box_x1_)
    x2=torch.min(box_x2,box_x2_)
    y1=torch.max(box_y1,box_y1_)
    y2=torch.min(box_y2,box_y2_)
    inner = (x2-x1).clamp(0)*(y2-y1).clamp(0)
    
    union = pred[2]*pred[3]+target[2]*target[3]-inner+1e-6
    
    return inner/union

In [ ]:
!pip install pascal-voc

In [ ]:
class YoloLoss(nn.Module):
    
    def __init__(self,S=7,B=2,C=20):
        self.S=S
        self.B=B
        self.C=C
        self.coord = 5
        self.noobj = 0.5
        self.mse = nn.MSE(reduction='sum')
        
    def forward(self,pred,target):
        pred = pred.reshape(-1,self.S,self.S,self.B*5+self.C)
        
        iou1 = iOU(pred[...,21:25],target[...,21:25])
        iou2 = iOU(pred[...,26:30],target[...,21:25])
        
        ious = torch.cat(iou1.unsqueeze(0),iou2.unsqueeeze(0),dim=0)
        iou_max_val,best_bbox = torch.max(ious,dim = 0)
        
        actual_box = target[...,20].unsqueeze(3)
        
        box_pred = actual_box & ((
        best_bbox * pred[...,26:30]+(1-best_bbox)*pred[...,21:25]
        ))
        

In [ ]:
!pip install dataset-convertor

In [ ]:
import os
import shutil
from glob import glob
from PIL import Image
from os.path import join as join_path
from datetime import datetime
from xml.etree import ElementTree as ET
import logging
from random import random

logging.basicConfig(format='%(levelname)s - %(asctime)s - %(name)s - %(message)s',
					datefmt='%d/%m/%Y %I:%M:%S %p',
					level=logging.INFO,
					handlers=[
        				logging.FileHandler('debug.log'),
        				logging.StreamHandler()])

class Convertor:
    '''
    Convertor class is a class for converting images and annotations
    from one format to another.
        
    Usage:
        # voc to yolo
        convertor = Convertor('data/VOC2007', 'outout/VOC2007_yolo', 'voc', 'yolo')
        convertor.voc2yolo()

        # voc to coco
        convertor = Convertor('data/VOC2007', 'outout/VOC2007_coco', 'voc', 'coco')
        convertor.voc2coco()

        # yolo to voc
        convertor = Convertor('data/yolo', 'outout/VOC2007', 'yolo', 'voc')
        convertor.yolo2voc()


    Parameters
    ----------
    input_folder : str
        Path to the input folder.
    output_folder : str
        Path to the output folder.
    input_format : str
        Input format of the images and annotations.
    output_format : str
        Output format of the images and annotations.

    Methods
    -------
    voc2yolo()
        Convert Pascal VOC format to Yolo5 format.

    yolo2voc()
        Convert Yolo5 format to Pascal VOC format.

    yolo2coco()
        Convert Yolo5 format to COCO format.
    
    coco2yolo()
        Convert COCO format to Yolo5 format.

    voc2coco()
        Convert Pascal VOC format to COCO format.
    
    coco2voc()
        Convert COCO format to Pascal VOC format.

    pascal2tfrecord()
        Convert Pascal VOC format to TFRecord format.
    
    coco2tfrecord()
        Convert COCO format to TFRecord format.

    yolo2tfrecord()
        Convert Yolo5 format to TFRecord format.


    # Pascal Voc - XML
    # Pascal VOC format is a XML file format for images and annotations.

    # Yolo5 - TXT
    # Yolo5 format is a text file format for images and annotations.

    # COCO - JSON
    # COCO format is a JSON file format for images and annotations.

    # TFRecord - TFRecord
    # TFRecord format is a TFRecord file format for images and annotations.

    '''
    def __init__(self, input_folder, output_folder):
        self.input_image_folder = join_path(input_folder, 'JPEGImages')
        self.input_annotation_folder = join_path(input_folder, 'Annotations')
        self.output_image_folder = join_path(output_folder, 'JPEGImages')
        self.output_annotation_folder = join_path(output_folder, 'Annotations')


    def voc2yolo(self):
        ''' Pascal Voc format to yolo txt format conversion '''

        os.makedirs(self.output_annotation_folder, exist_ok=True)
        os.makedirs(self.output_annotation_folder+'/train', exist_ok=True)
        os.makedirs(self.output_annotation_folder+'/test', exist_ok=True)
        #shutil.copytree(self.input_image_folder, self.output_image_folder)

        for xml_file in glob(join_path(self.input_annotation_folder, '*.xml')):

            tree = ET.parse(xml_file)
            root = tree.getroot()

            file_name = root.find('filename').text
            width = float(root.find('size').find('width').text)
            height = float(root.find('size').find('height').text)
            spiltrand = random()
            if spiltrand < 0.8:
                output_txt_file = (self.output_annotation_folder+'/train/'+file_name.split('.')[0] + '.txt')
            else:
                output_txt_file = (self.output_annotation_folder+'/test/'+file_name.split('.')[0] + '.txt')
            for obj in root.iter('object'):
                xmin = float(obj.find('bndbox').find('xmin').text)
                ymin = float(obj.find('bndbox').find('ymin').text)
                xmax = float(obj.find('bndbox').find('xmax').text)
                ymax = float(obj.find('bndbox').find('ymax').text)
                label = obj.find('name').text

                # conver xmin ymin xmax ymax to x y w h
                x = (xmin + xmax) / width
                y = (ymin + ymax) / height
                w = (xmax - xmin) / width
                h = (ymax - ymin) / height

                #print(output_txt_file)
                with open(output_txt_file, 'a') as f:
                    f.write('{} {} {} {} {}\n'.format(label, str(x), str(y), str(w), str(h)))
            
    def readtxt(self): 
        for filepath,dirnames,filenames in os.walk(self.output_annotation_folder):
            for filename in filenames:
                with open(dirnames+'/'+filenames,'a') as f:
                    print(f.read())


    

In [ ]:
from random import random
random()

In [ ]:
def readtxt(output_annotation_folder): 
    print(output_annotation_folder)
    for filepath,dirnames,filenames in os.walk(output_annotation_folder):
        for filename in filenames:
            with open(filepath+'/'+filename,'r') as f:
                print(f.read())

In [ ]:
#from convert import Convertor

con = Convertor(input_folder='/kaggle/input/pascal-voc-2012/VOC2012/',output_folder='/kaggle/working/yolov1/')
con.voc2yolo()
#readtxt('/kaggle/working/yolov1/Annotation')

In [ ]:
readtxt('/kaggle/working/yolov1/Annotations/')

In [ ]:
from pathlib import Path
import pascal

from pascal import PascalVOC

ds = Path("/kaggle/input/pascal-voc-2012/VOC2012/Annotations/")
print(ds)
label_map = {
    "plate": 0,
    "other": 1,
    "taxi": 2,
    "standard": 3
}

print(ds.glob("*.xml"))

for file in ds.glob("*.xml"):
    print(file)
    ann = PascalVOC.from_xml(file)
    yolo = ann.to_yolo(label_map)
    out_name = f"{file.stem}.txt"
    with open(out_name, "w") as f:
        f.write(yolo)

In [ ]:
os.path

In [ ]:
import torch
x=torch.tensor([[-1,2,3,4],[2,1,3,4],[5,6,7,8]])
x[...,0:1]
a,b=torch.max(x,dim=1)
print(Path(''))